<a href="https://colab.research.google.com/github/DSWoodin/Pythonworkspace/blob/main/ProDS/%EB%AA%A8%ED%8F%89%EA%B7%A0%EB%B9%84%EA%B5%90%EC%97%90%EA%B4%80%ED%95%9C%EA%B0%80%EC%84%A4%EA%B2%80%EC%A0%95_OnewayANOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from scipy.stats import f_oneway
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
df = pd.read_csv('diamonds.csv')
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [3]:
df['color'].unique()

array(['E', 'I', 'J', 'H', 'F', 'G', 'D'], dtype=object)

In [4]:
stat, p = f_oneway(df.loc[df['color'] == 'E', 'price'],
                   df.loc[df['color'] == 'I', 'price'],
                   df.loc[df['color'] == 'J', 'price'])
print(round(stat, 3))
print(round(p, 3))

621.846
0.0


In [6]:
df.groupby('color')['price'].mean().reset_index()

,color,price
0,D,3169.954096
1,E,3076.752475
2,F,3724.886397
3,G,3999.135671
4,H,4486.669196
5,I,5091.874954
6,J,5323.818020


In [7]:
bike = pd.read_csv('bike.csv')
bike.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [8]:
model = ols(formula = 'temp ~ season', data = bike).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
season,1.0,44221.657301,44221.657301,780.591754,5.879560e-166
Residual,10884.0,616594.417651,56.651453,NaN,NaN


In [9]:
# season이 명목형 변수라고 알려줘야 함 C()
model = ols(formula = 'temp ~ C(season)', data = bike).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(season),3.0,412885.270005,137628.423335,6040.687453,0.0
Residual,10882.0,247930.804947,22.783570,NaN,NaN


In [11]:
result = pairwise_tukeyhsd(bike['temp'], bike['season'])
print(result)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj  lower    upper   reject
-----------------------------------------------------
     1      2   10.293   0.0   9.9598  10.6262   True
     1      3  16.2586   0.0  15.9254  16.5918   True
     1      4   4.1187   0.0   3.7856   4.4519   True
     2      3   5.9656   0.0   5.6339   6.2974   True
     2      4  -6.1742   0.0   -6.506  -5.8425   True
     3      4 -12.1399   0.0 -12.4716 -11.8081   True
-----------------------------------------------------


### 문제 1: 계절별 기온 평균을 분산분석 시 검정통계량은?

In [13]:
df = pd.read_csv('bike.csv')
df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [15]:
formula = 'temp ~ C(season)' # C() 범주형 변수임을 명시
lm = ols(formula, df).fit()
anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
C(season),3.0,412885.270005,137628.423335,6040.687453,0.0
Residual,10882.0,247930.804947,22.783570,NaN,NaN


### 문제 2: 요일별 registered의 평균을 사후검정 했을 때 평균이 유의미하게 차이 나지 않는 조합은 총 몇 개인가?

In [16]:
df = pd.read_csv('bike.csv')
df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [19]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['wday'] = df['datetime'].dt.weekday
df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,weekday,wday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,5,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,5,5


In [20]:
print(pairwise_tukeyhsd(df['registered'], df['wday']))

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     0      1   6.1979  0.913   -9.719  22.1148  False
     0      2    5.343 0.9559 -10.5429  21.2289  False
     0      3  12.7424 0.2132  -3.1384  28.6232  False
     0      4   6.2956 0.9074  -9.6473  22.2386  False
     0      5 -27.5063    0.0 -43.3093 -11.7034   True
     0      6 -36.7583    0.0 -52.5736 -20.9429   True
     1      2  -0.8549    1.0 -16.7718   15.062  False
     1      3   6.5445 0.8895  -9.3673  22.4562  False
     1      4   0.0977    1.0 -15.8761  16.0715  False
     1      5 -33.7042    0.0 -49.5383 -17.8702   True
     1      6 -42.9562    0.0 -58.8026 -27.1097   True
     2      3   7.3994 0.8159  -8.4814  23.2802  False
     2      4   0.9526    1.0 -14.9903  16.8956  False
     2      5 -32.8493    0.0 -48.6523 -17.0464   True
     2      6 -42.1013    0.0 -57.9166 -26.2859   True
     3    